## Overview

In this notebook, we load a saved model trained in `example_model_training.ipynb`, and use it to perform predictive 
inference on events of interest from frames from a new video it has not been trained on.